In [1]:
import numpy as np
import math 
import pandas as pd
import time
import plotly.graph_objects as go
import random
import plotly.express as px

### Function

In [2]:
#Stableswap function that outputs token supply of Y, given X, A factor, and D (the sum of the two tokens when in parity)
def sampleStable(x = 50,A = 10,D = 100, n = 2):
    if A == 0:
        return D-x
    if x<=D:
        y = (-(A*4)*(x**2) + A*D*x*n**n - D*x + math.sqrt(((A*n**n)*(x**2) - A*D*x*n**n + D*x)**2 + 4*A*x*n**n*((D**(n+1))/n**n)))/(2*A*x*n**n)
        return y
    if x>D:
        print('D is greater than X, shit aint gonna work')

#given the usdc side, and the price, output the bean and usdc in pool at price, and total assets (D) when in parity
def stableSwap(usdc,pri,A = 10):
    if A == 0:
        #special case for XYK
        return [usdc/pri, usdc, 2*math.sqrt(usdc*usdc/pri)]
    inital_D = usdc*2
    inital_bean = usdc
    test = 1
    #simple for loop that estimates condictions
    for i in np.linspace(0,1,1000):
        #TLDR: trying to find D: we start with the inital D, and increase it stepwise until we find the correct D, which then we can find the bean assoicated with it) 
        temp_D = inital_D/(1+i)
        amtOut = sampleStable(usdc,A,temp_D,2) - sampleStable(usdc + 1,A,temp_D,2)
        #since this is a guesstimate (an accurate one, but still), we simply compare them lmao
        if test > abs(pri - (1/amtOut)):
            test = abs(pri - (1/amtOut))
        else:
            break
    newBean = sampleStable(usdc,A,temp_D,2)
    D = temp_D
    return [newBean,usdc,D]

#given bean,usdc,D size, and A factor, output price 
def getPrice(usdc,D, amtIn = 1,A=10):
    if A == 0:
        y = D - usdc
        k = usdc * y
        usdc = usdc + amtIn
        new_y = k/usdc
        return amtIn/(y-new_y)

    amtOut =sampleStable(usdc,A,D,2) - sampleStable(usdc + amtIn,A,D,2)
    price = amtIn/amtOut
    return price

#given bean,usdc,D, gives the delta (mints needed to get back to peg)
def getDelta(bean,usdc,D):
    target = D/2
    delta = target - bean
    return delta 
# the amount of bean in the pool that can be acquired with 1% slippage given A factor
# assumes slippage occurs at parity 
def slippageFinder(slippage, A):
    priceblah = 0
    i = 0
    # estimate the % of bean you can obtain from the pool by comparing the price you got
    while  abs(priceblah - (1 + slippage)) > 0.0001:
        i = i + 1
        priceblah = getPrice(1000,2000,i,A)
    New_Curve_Price = getPrice(1000 + i,2000,1,A)
    
    
    return [A,i/1000,New_Curve_Price]

# not used 

#very rough calc, just takes the current mint, and finds the rough apr by exploiating it to a year 
def roughAPR(mints,stalk):
    beanPerStalk = mints/stalk
    APR = beanPerStalk*24*365

#this simulates a cycle of demand (there may be times where there is a lot of mints, and times where there is no mints)
def cycleMode(season,avgPrice): 
    price = (math.sin(2* season/24)/25 + avgPrice) + (math.sin(50*season*(random.randint(0, 100)/100))/100)
    return price                

### Examples

In [3]:
# Sample to calculate how many of token Y given quantity of token X, and total assets.
# Inital conditions: 
# Total sum of assets when in parity: 100
# Amount of token X: 50
# A factor: 10
print(sampleStable())
# Gets the price of a pool, 
# Inital conditions: 1000 USDC in the pool 
# 2000 tokens when in parity 
# Trading 1 usdc in
# A factor of 10
print(getPrice(1000,2000,1,10))
# Outputs the bean and usdc in a pool at 1.02
# assumes 10m USDC, A factor of 10
print(stableSwap(10_000_000,1.02,A = 10))

50.0
1.0000476190907506
[6735384.559373089, 10000000, 16719665.271966526]


In [4]:
# Dataframe of Balances of Bean/USDC at 1.01 at A= 0-10 (index = A)
# Assumes USDC in pool is always 10_000
# slippage = 1 means 1% slippage

def slippageDeltaConstantUSDC(slippage=1):
    df1=pd.DataFrame(columns = ['bean','usdc','delta_B'])
    price = 1 + slippage/100
    #determine the liquidity pool size 
    for A_factor in range(0,11):
        var=stableSwap(10000,price,A_factor)[:-1]
        var.append(getDelta(stableSwap(10000,price,A_factor)[0],10000,stableSwap(10000,price,A_factor)[2]))
        df1.loc[len(df1.index)] = var
    
    # data_perc = df.divide(df.sum(axis=1), axis=0)
    # fig = data_perc.plot.area(y=['usdc','bean'])
    # data_perc
    return df1

In [14]:
df1 = slippageDeltaConstantUSDC()
df1

,bean,usdc,delta_B
0,9900.990099,10000.0,49.381803
1,9685.566254,10000.0,156.798278
2,9493.996998,10000.0,252.344465
3,9306.130949,10000.0,346.042964
4,9121.861601,10000.0,437.947011
5,8958.961909,10000.0,519.216459
6,8781.248731,10000.0,607.849013
7,8606.846646,10000.0,694.829332
8,8435.664302,10000.0,780.202856
9,8284.235096,10000.0,855.746606


In [6]:
# Balances of Bean/USDC at 1.01 at A = 0-10 (index = A)
# Assumes D is constant instead (D = 20000)

def slippageDeltaConstantD(slippage=1):
    df2=pd.DataFrame(columns = ['bean','usdc','delta_B'])
    D= 20000
    usdc=D/2
    price = 1.01
    #determine the liquidity pool size 
    for A_factor in range(0,11):
        usdc=D/2
        test = 1
        while test > abs(price - getPrice(usdc,D,1,A_factor)):
            test = abs(price - getPrice(usdc,D,1,A_factor))
            usdc +=1
        bean = sampleStable(usdc,A_factor,D,2)
        df2.loc[len(df2.index)] = [usdc,bean,getDelta(bean,usdc,D)]
    return(df2)


In [15]:
df2 = slippageDeltaConstantD()
df2

,bean,usdc,delta_B
0,10050.0,9950.000000,50.000000
1,10150.0,9850.746343,149.253657
2,10250.0,9751.244274,248.755726
3,10349.0,9652.732917,347.267083
4,10448.0,9554.222488,445.777512
5,10546.0,9456.703353,543.296647
6,10644.0,9359.185694,640.814306
7,10742.0,9261.669781,738.330219
8,10838.0,9166.136050,833.863950
9,10933.0,9071.594663,928.405337


In [8]:
def beansPerSlippage(slippage = 1):
# dataframe to show amount of beans that can be bought in one trade, with 1% slippage
# note that new price will be higher than the purchase price
# slippage = 1 means 1% slippage
    df3=pd.DataFrame(columns = ['A factor','% Bean in Pool','Virtual Price',])

    #determine the liquidity pool size 
    for A_factor in range(1,11):
        df3.loc[len(df3.index)] = slippageFinder(slippage/100,A_factor)
    return df3
    

In [9]:
df3 = beansPerSlippage()
df3

,A factor,% Bean in Pool,Virtual Price
0,1.0,0.030,1.020454
1,2.0,0.050,1.020368
2,3.0,0.070,1.020392
3,4.0,0.089,1.020242
4,5.0,0.108,1.020214
5,6.0,0.127,1.020267
6,7.0,0.146,1.020380
7,8.0,0.165,1.020546
8,9.0,0.183,1.020631
9,10.0,0.201,1.020777


In [276]:
fig = px.line(df3, x='A factor' , y = '% Bean in Pool', title= "Amount of beans 1 trade can obtain with 1% slippage (assuming parity)")
fig.show()

In [10]:
# the price of a pool given a delta, from A= 1-10
# inputs are max price, and an array of A factors
def poolPriceDelta(price=1.5,A_factor=[0,1,2,3,4,5,6,7,8,9,10]):
    D=20000
    usdc=D/2
    df4=pd.DataFrame(columns = ['A Factor','Delta B','Price'])
    for A_factor in np.linspace(0,10,11):
        for price in np.linspace(1,2,10):
            usdc=D/2
            test = 1
            while test > abs(price - getPrice(usdc,D,1,A_factor)):
                test = abs(price - getPrice(usdc,D,1,A_factor))
                usdc +=1
            bean = sampleStable(usdc,A_factor,D,2)
            df4.loc[len(df4.index)] = [A_factor,getDelta(bean,usdc,D),price]
    return df4

In [11]:
df4 = poolPriceDelta()
df4

,A Factor,Delta B,Price
0,0.0,1.000000,1.000000
1,0.0,527.000000,1.111111
2,0.0,1001.000000,1.222222
3,0.0,1429.000000,1.333333
4,0.0,1819.000000,1.444444
...,...,...,...
105,10.0,8002.686673,1.555556
106,10.0,8183.561192,1.666667
107,10.0,8324.229486,1.777778
108,10.0,8437.790625,1.888889


In [12]:
fig = px.line(df4, x="Delta B", y="Price", color='A Factor',title = 'Price vs. Delta given A factor')
fig.show()

In [ ]:
df1.to_csv("slippageDeltaConstantUSDC.csv")
df2.to_csv("slippageDeltaConstantD.csv")
df3.to_csv("beansPerSlippage.csv")
df4.to_csv("poolPriceDelta.csv")
